<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/System_eng_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hh")